# Installing Package for Building Image on Sagemaker Studio

In [3]:
!pip install sagemaker-studio-image-build

  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13454 sha256=dc6e33ae505b4cd7b8eadbcf5e3642c991be7535a2dac758102420d513b98c84
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/69/7b/d1/1318b1530ee5322c9be00f206badea11b5148626ef58c0e0dc
Successfully built sagemaker-studio-image-build


# Build Image and Push to ECR

In [23]:
%%sh

# The name of our algorithm
spec_role=ins-dev-role-sagemaker
algorithm_name=ins-demo-sagemaker-case02
image_ver=v1

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-southeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-southeast-1}

# fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${image_ver}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR with the full name.
# sm-docker build -t ${fullname} . --role ${spec_role} --repository ${algorithm_name}:${image_ver}
sm-docker build . --role ${spec_role} --repository ${algorithm_name}:${image_ver}

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
....................[Container] 2024/07/25 07:33:38.175732 Running on CodeBuild On-demand

[Container] 2024/07/25 07:33:38.175749 Waiting for agent ping
[Container] 2024/07/25 07:33:38.278949 Waiting for DOWNLOAD_SOURCE
[Container] 2024/07/25 07:33:48.958420 Phase is DOWNLOAD_SOURCE
[Container] 2024/07/25 07:33:49.010840 CODEBUILD_SRC_DIR=/codebuild/output/src3447363066/src
[Container] 2024/07/25 07:33:49.011785 YAML location is /codebuild/output/src3447363066/src/buildspec.yml
[Container] 2024/07/25 07:33:49.014876 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2024/07/25 07:33:49.015065 Processing environment variables
[Container] 2024/07/25 07:33:49.084113 No runtime version selected in buildspec.
[Container] 2024/07/25 07:33:49.175997 Moving to dire

# Let's Play!

## Set Environment

In [27]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

role

'arn:aws:iam::891377385085:role/service-role/AmazonSageMaker-ExecutionRole-20240605T164521'

## Create Seassion

In [28]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload Data

In [29]:
# S3 prefix
bucket_name = "ins-demo-bucket"
prefix = "demo-sagemaker/case02"
WORK_DIRECTORY = "data"

data_location = sess.upload_data(WORK_DIRECTORY, bucket=bucket_name, key_prefix=prefix)

## Create an Estimator and Fit the Model

In [30]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

algorithm_name="ins-demo-sagemaker-case02"
image_ver="v1"
image = "{}.dkr.ecr.{}.amazonaws.com/{}:{}".format(account, region, algorithm_name, image_ver)

tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/{}/output".format(bucket_name, prefix),
    sagemaker_session=sess,
)

tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: ins-demo-sagemaker-case02-2024-07-25-07-48-02-333


2024-07-25 07:48:02 Starting - Starting the training job...
2024-07-25 07:48:20 Starting - Preparing the instances for training...
2024-07-25 07:49:03 Downloading - Downloading the training image
2024-07-25 07:49:03 Training - Training image download completed. Training in progress.
2024-07-25 07:49:03 Uploading - Uploading generated training model.Starting the training.
Training complete.

2024-07-25 07:49:15 Completed - Training job completed
Training seconds: 34
Billable seconds: 34


## Deploy Model

In [49]:
from sagemaker.serializers import CSVSerializer

predictor = tree.deploy(1, "ml.m4.xlarge", serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: ins-demo-sagemaker-case02-2024-07-25-08-01-16-164
INFO:sagemaker:Creating endpoint-config with name ins-demo-sagemaker-case02-2024-07-25-08-01-16-164
INFO:sagemaker:Creating endpoint with name ins-demo-sagemaker-case02-2024-07-25-08-01-16-164


----!

## Testing Model

In [50]:
df_sample = pd.read_csv("data/iris.csv", header=None)
df_sample.sample(3)

,0,1,2,3,4
80,versicolor,5.5,2.4,3.8,1.1
124,virginica,6.7,3.3,5.7,2.1
31,setosa,5.4,3.4,1.5,0.4


In [51]:
# drop the label column in the training set
df_sample.drop(df_sample.columns[[0]], axis=1, inplace=True)
df_sample.sample(3)

,1,2,3,4
133,6.3,2.8,5.1,1.5
109,7.2,3.6,6.1,2.5
5,5.4,3.9,1.7,0.4


In [52]:
import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = df_sample.iloc[indices[:-1]]

test_data.head(5)

,1,2,3,4
40,5.0,3.5,1.3,0.3
41,4.5,2.3,1.3,0.3
42,4.4,3.2,1.3,0.2
43,5.0,3.5,1.6,0.6
44,5.1,3.8,1.9,0.4


In [53]:
test_data.iloc[0, :].values

array([5. , 3.5, 1.3, 0.3])

In [54]:
print(predictor.predict(test_data.values).decode("utf-8"))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



## Cleanup

In [ ]:
# sess.delete_endpoint(predictor.endpoint)

# Installing Package for Building Image on Sagemaker Studio

In [3]:
!pip install sagemaker-studio-image-build

  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13454 sha256=dc6e33ae505b4cd7b8eadbcf5e3642c991be7535a2dac758102420d513b98c84
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/69/7b/d1/1318b1530ee5322c9be00f206badea11b5148626ef58c0e0dc
Successfully built sagemaker-studio-image-build


# Build Image and Push to ECR

In [23]:
%%sh

# The name of our algorithm
spec_role=ins-dev-role-sagemaker
algorithm_name=ins-demo-sagemaker-case02
image_ver=v1

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-southeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-southeast-1}

# fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${image_ver}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR with the full name.
# sm-docker build -t ${fullname} . --role ${spec_role} --repository ${algorithm_name}:${image_ver}
sm-docker build . --role ${spec_role} --repository ${algorithm_name}:${image_ver}

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
....................[Container] 2024/07/25 07:33:38.175732 Running on CodeBuild On-demand

[Container] 2024/07/25 07:33:38.175749 Waiting for agent ping
[Container] 2024/07/25 07:33:38.278949 Waiting for DOWNLOAD_SOURCE
[Container] 2024/07/25 07:33:48.958420 Phase is DOWNLOAD_SOURCE
[Container] 2024/07/25 07:33:49.010840 CODEBUILD_SRC_DIR=/codebuild/output/src3447363066/src
[Container] 2024/07/25 07:33:49.011785 YAML location is /codebuild/output/src3447363066/src/buildspec.yml
[Container] 2024/07/25 07:33:49.014876 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2024/07/25 07:33:49.015065 Processing environment variables
[Container] 2024/07/25 07:33:49.084113 No runtime version selected in buildspec.
[Container] 2024/07/25 07:33:49.175997 Moving to dire

# Let's Play!

## Set Environment

In [27]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

role

'arn:aws:iam::891377385085:role/service-role/AmazonSageMaker-ExecutionRole-20240605T164521'

## Create Seassion

In [28]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload Data

In [29]:
# S3 prefix
bucket_name = "ins-demo-bucket"
prefix = "demo-sagemaker/case02"
WORK_DIRECTORY = "data"

data_location = sess.upload_data(WORK_DIRECTORY, bucket=bucket_name, key_prefix=prefix)

## Create an Estimator and Fit the Model

In [30]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

algorithm_name="ins-demo-sagemaker-case02"
image_ver="v1"
image = "{}.dkr.ecr.{}.amazonaws.com/{}:{}".format(account, region, algorithm_name, image_ver)

tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/{}/output".format(bucket_name, prefix),
    sagemaker_session=sess,
)

tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: ins-demo-sagemaker-case02-2024-07-25-07-48-02-333


2024-07-25 07:48:02 Starting - Starting the training job...
2024-07-25 07:48:20 Starting - Preparing the instances for training...
2024-07-25 07:49:03 Downloading - Downloading the training image
2024-07-25 07:49:03 Training - Training image download completed. Training in progress.
2024-07-25 07:49:03 Uploading - Uploading generated training model.Starting the training.
Training complete.

2024-07-25 07:49:15 Completed - Training job completed
Training seconds: 34
Billable seconds: 34


## Deploy Model

In [49]:
from sagemaker.serializers import CSVSerializer

predictor = tree.deploy(1, "ml.m4.xlarge", serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: ins-demo-sagemaker-case02-2024-07-25-08-01-16-164
INFO:sagemaker:Creating endpoint-config with name ins-demo-sagemaker-case02-2024-07-25-08-01-16-164
INFO:sagemaker:Creating endpoint with name ins-demo-sagemaker-case02-2024-07-25-08-01-16-164


----!

## Testing Model

In [50]:
df_sample = pd.read_csv("data/iris.csv", header=None)
df_sample.sample(3)

,0,1,2,3,4
80,versicolor,5.5,2.4,3.8,1.1
124,virginica,6.7,3.3,5.7,2.1
31,setosa,5.4,3.4,1.5,0.4


In [51]:
# drop the label column in the training set
df_sample.drop(df_sample.columns[[0]], axis=1, inplace=True)
df_sample.sample(3)

,1,2,3,4
133,6.3,2.8,5.1,1.5
109,7.2,3.6,6.1,2.5
5,5.4,3.9,1.7,0.4


In [52]:
import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = df_sample.iloc[indices[:-1]]

test_data.head(5)

,1,2,3,4
40,5.0,3.5,1.3,0.3
41,4.5,2.3,1.3,0.3
42,4.4,3.2,1.3,0.2
43,5.0,3.5,1.6,0.6
44,5.1,3.8,1.9,0.4


In [53]:
test_data.iloc[0, :].values

array([5. , 3.5, 1.3, 0.3])

In [54]:
print(predictor.predict(test_data.values).decode("utf-8"))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



## Cleanup

In [ ]:
# sess.delete_endpoint(predictor.endpoint)